In [1]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
import os
import random

import cv2
import keras.utils
import numpy as np
from keras.layers import Input, Dense,Flatten, Concatenate
from keras.models import Model
from keras.ops import empty

keras.utils.set_random_seed(42)
np.random.seed(42)
random.seed(42)


###TRAINING VARIABLES

sample_count = 79999
epoch_count = 100
batch_size = 7
validation_count = 20000
initial_learning_rate=.001
decay_steps=8888
decay_rate=0.97
dataset_name = "SuToBoCM"



inp1 = Input(shape=(320, 240, 1))
inp2 = Input(shape=(320, 240, 1))
empty = keras.ops.empty(shape=(batch_size, 320, 240, 1))

concat = Concatenate(axis=-1)([inp1, inp2])
concat = Concatenate(axis=-1)([concat, empty])


resnet = keras.applications.ResNet50V2(
    include_top=False,
    weights=None,
    input_tensor=concat,
    input_shape=(320, 240, 3),
    pooling=None,
    classes=1000,
    classifier_activation="relu",
)

flat = Flatten()(resnet.output)
dense = Dense(128, activation="relu")(flat)

chest_out = Dense(1, activation='linear', name='chest_out')(dense)
waist_out = Dense(1, activation='linear', name='waist_out')(dense)
pelvis_out = Dense(1, activation='linear', name='pelvis_out')(dense)
# neck_out = Dense(1, activation='linear', name='neck_out')(dense)
bicep_out = Dense(1, activation='linear', name='bicep_out')(dense)
thigh_out = Dense(1, activation='linear', name='thigh_out')(dense)
# knee_out = Dense(1, activation='linear', name='knee_out')(dense)
shoulder_to_wrist_out = Dense(1, activation='linear', name='shoulder_to_wrist_out')(dense)
leg_out = Dense(1, activation='linear', name='leg_out')(dense)
calf_out = Dense(1, activation='linear', name='calf_out')(dense)
# head_out = Dense(1, activation='linear', name='head_out')(dense)
wrist_out = Dense(1, activation='linear', name='wrist_out')(dense)
arm_out = Dense(1, activation='linear', name='arm_out')(dense)
shoulder_to_shoulder_out = Dense(1, activation='linear', name='shoulder_to_shoulder_out')(dense)
# torso_out = Dense(1, activation='linear', name='torso_out')(dense)
inner_leg_out = Dense(1, activation='linear', name='inner_leg_out')(dense)


model = Model(
    inputs=[inp1, inp2],
    outputs=[
        chest_out,
        waist_out,
        pelvis_out,
        # neck_out,
        bicep_out,
        thigh_out,
        # knee_out,
        shoulder_to_wrist_out,
        leg_out,
        calf_out,
        # head_out,
        wrist_out,
        arm_out,
        shoulder_to_shoulder_out,
        # torso_out,
        inner_leg_out
    ],
    name='ResNet50V2'
)




folder_path = f"{dataset_name} - {sample_count}s{epoch_count}e{validation_count}v {"" if initial_learning_rate == 0.001 else initial_learning_rate + "ilr"}{decay_steps}s{decay_rate}d"

try:
    os.mkdir(os.path.join("./logs/" + folder_path))
except OSError as error:
    print(error)
    

#model.save_weights("default.weights.h5")

# model.summary()
# 
from keras.utils import plot_model

plot_model(model, to_file='ResNet50+Side.png', show_shapes=True, show_layer_names=True, show_layer_activations=True)

In [ ]:
from DatasetLoaderClass import DatasetLoader
quickTrain = {
                'image_location': "./Surreact-APose/train/images_front/", 
                'data_location': "./Surreact-APose/train/measurementsCM/", 
                'sample_count': sample_count, 
                'batch_size': batch_size, 
                'seed':  69,#np.random.randint(0, 10000), 
                'use_memory': False,
                'random_sample': False,
                'shuffle': True}

quickValidate = {
                'image_location': "./Surreact-APose/test/images_front/",
                'data_location': "./Surreact-APose/test/measurementsCM/",
                'sample_count': validation_count,
                'batch_size': batch_size,
                'seed': 69, #np.random.randint(0, 10000),
                'use_memory': False,
                'random_sample': False}

train_generator = DatasetLoader(**quickTrain)
validation_generator = DatasetLoader(**quickValidate)

In [ ]:
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, EarlyStopping
from keras.losses import MeanSquaredError


lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True)

stop_no_improvement = keras.callbacks.EarlyStopping(monitor="loss", patience=3)


model.compile(
    optimizer=Adam(learning_rate=lr_schedule),
    loss={
        'chest_out': MeanSquaredError(),
        'waist_out': MeanSquaredError(),
        'pelvis_out': MeanSquaredError(),
        # 'neck_out': MeanSquaredError(),
        'bicep_out': MeanSquaredError(),
        'thigh_out': MeanSquaredError(),
        # 'knee_out': MeanSquaredError(),
        'shoulder_to_wrist_out': MeanSquaredError(),
        'leg_out': MeanSquaredError(),
        'calf_out': MeanSquaredError(),
        # 'head_out': MeanSquaredError(),
        'wrist_out': MeanSquaredError(),
        # 'arm_out': MeanSquaredError(),
        'shoulder_to_shoulder_out': MeanSquaredError(),
        # 'torso_out': MeanSquaredError(),
        # 'inner_leg_out': MeanSquaredError()
    },
    metrics={
        'chest_out': ['mae'],
        'waist_out': ['mae'],
        'pelvis_out': ['mae'],
        # 'neck_out': ['mae'],
        'bicep_out': ['mae'],
        'thigh_out': ['mae'],
        # 'knee_out': ['mae'],
        'shoulder_to_wrist_out': ['mae'],
        'leg_out': ['mae'],
        'calf_out': ['mae'],
        # 'head_out': ['mae'],
        'wrist_out': ['mae'],
        # 'arm_out': ['mae'],
        'shoulder_to_shoulder_out': ['mae'],
        # 'torso_out': ['mae'],
        # 'inner_leg_out': ['mae'],
    },
)



checkpoint_filepath = './models/ResNet50V2.model.keras'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='loss',
    mode='min',
    save_best_only=True)

model.fit(
    x=train_generator,
    validation_data=validation_generator,
    callbacks=[
        
        TensorBoard(write_graph=True,log_dir="./logs/"+folder_path),
        model_checkpoint_callback,
    ],
    batch_size=batch_size,
    epochs=epoch_count,
    verbose=1,
)

model.save("./models/full/ResNet50V2.keras")
# 
# for i in range(10000):
#     image = cv2.imread(f"../Surreact-APose/train/imgs_nobg_frontEDITED/Avatar_{6969+i:06d}.png", cv2.IMREAD_GRAYSCALE)
# 
#     print(model.predict(np.array([image])))
#     